# HP Bars

`HP Bars` as we call them are just really simple colorbars embedded in tabs that show the categorical breakdown of the data contained within.

Imagine we have a dashboard with multiple tabs of data. Each tab contains (multiple) large tables of data where each cell in the tables are highlighted with different colors based on some criteria. For large numbers of tabs and tables, simply viewing the data may not readily give a summary view, and you may need to look at multiple tabs/tables before finding something interesting.

Additional panes can be added to the interface that provides this information, but this is real estate in the UI that is now being taken up by something. Having a really simple indicator that embeds additional information without requiring more space within your UI can help give the same effect as a tabular breakdown of everything included at the high level.

This notebook will show how to embed HP Bars into components to help give some additional insight without cluttering your dashboard with more widgets/components.

In [1]:
import numpy as np
import pandas as pd
import panel as pn

print('numpy =', np.__version__)
print('pandas =', pd.__version__)
print('panel =', pn.__version__)

numpy = 1.20.2
pandas = 1.2.4
panel = 0.11.3


In [2]:
# here we are going to simulate a large collection of data - each tab of data will be identical in structure,
# but the data itself will be randomly generated. to simulate what I do at work, we show "current" and "previous"
# tables along with a third "delta" table. we will be generating 5 tabs of 3 tables
tab_data = [
    [pd.DataFrame({
        'q_data': np.random.randint(0, 100, 100),
        'w_data': np.random.randint(0, 100, 100),
        'e_data': np.random.randint(0, 100, 100),
        'r_data': np.random.randint(0, 100, 100),
        't_data': np.random.randint(0, 100, 100),
        'y_data': np.random.randint(0, 100, 100),
    }) for _ in range(2)] for __ in range(5)
]

# artificially modify tables to show some exaggerated results
tab_data[3][1] *= 2
tab_data[4][1] //= 2

# create delta tables
for td in tab_data:
    td.append(td[0]-td[1])

## Defining the HP Bars

This is where the magic happens! We are generating a custom CSS to modify the bokeh tab widget (which is wrapped by panel). We iterate over each tab and examine the 3rd table (the deltas). For this example we break things down so that we count deltas less than -10, between -10 and 10, and greater than 10 - these will respectively be colored red, green, and blue. Once we count each color category, we turn them into percentages and then build the custom CSS.

The main bit here is (mind the Python formatting curly-braces):

```css
background-image: linear-gradient(90deg, red {r}%, green {r}% {r+g}%, blue {r+g}%);
```

This tells your browser to fill the space with a gradient. We accumulate the percentages and use them to specify how the colorbar should be broken up. By repeating percentages we create hard divisions in the gradient. I would suggest checkout out the documentation either at [W3](https://www.w3schools.com/css/css3_gradients.asp) or [Mozilla](https://developer.mozilla.org/en-US/docs/Web/CSS/gradient/linear-gradient()) for all of options here.

The code below uses a loop to create CSS for each tab. Luckily, we can use CSS selectors to specify which tab we are working on - `.bk.bk-tab:nth-of-type` allows us to index each tab individually. We also add in `.panel-hp-bar`, our custom name, so that we can control which tabs we are adding the HP Bars to.

The rest of the CSS itself is just sizing and positioning so that the bar is below the label of the tab (otherwise the entire space would be colored with the gradient, and this is likely not desired).

In [3]:
raw_css = []
for idx, dfs in enumerate(tab_data, start=1):
    df = dfs[2]
    
    r = 100 * (df < -10.0).sum().sum() / df.size
    g = 100 * (np.abs(df)<=10.0).sum().sum() / df.size
    b = 100 * (df > 10.0).sum().sum() / df.size
        
    raw_css.append(f'''.panel-hp-bar .bk.bk-tab:nth-of-type({idx}) {{                   # select our tabs
    background-image: linear-gradient(90deg, red {r}%, green {r}% {r+g}%, blue {r+g}%); # apply the colorbar
    background-position: 0% 100%;                                                       # send the colorbar to the bottom
    background-size: 100% 30%;                                                          # squish the colorbar
    background-repeat: no-repeat;                                                       # ensure it doesn't repeat (it will still try to fill the space otherwise)
}}''')

# tell panel to use the css that we just defined    
pn.extension(raw_css=raw_css)

## Update the Tables

Here we color the cells per our coloring schema and then define some additional properties to make the tables looks prettier. By default `pandas` detaframes are rendered nicely in the browser, but once you start working with their `Styler` objects directly, the style is stripped down and they look very rigid (we just make them look nice again!).

In [4]:
def apply_bg_color(el):
    c = 'text-align:center !important;color:white;'
    if el < -10.0:
        c += 'background-color:red;'
    elif np.abs(el) <= 10.0:
        c += 'background-color:green;'
    else:
        c += 'background-color:blue;'
    return c

# this is all optional, but this helps to keep our tables looking nice
table_styles = [
    {
        'selector': 'th',
        'props': [
            ('text-align', 'left'),
            ('padding', '3px'),
            ('font-size', '12pt'),
            ('white-space', 'nowrap'),
            ('border', '0.5pt solid black'),
        ]
    },
    {
        'selector': 'td',
        'props': [
            ('text-align', 'right'),
            ('padding', '3px'),
            ('font-size', '9pt'),
            ('white-space', 'nowrap'),
            ('border', '0.5pt solid black'),
        ]
    },
    {
        'selector': 'tbody tr:nth-child(odd)',
        'props': [
            ('background-color', 'rgba(230,230,230,1)')
        ]
    }
]

# styled_data will be our final set of tables to visualize
styled_data = []
for td in tab_data:
    styled_data.append([
        td[0].style.set_table_styles(table_styles).hide_index().set_caption("Current"),
        td[1].style.set_table_styles(table_styles).hide_index().set_caption("Previous"),
        td[2].style.applymap(apply_bg_color).set_table_styles(table_styles).hide_index().set_caption("Delta"),        
    ])

## Create the Tabs

Lastly we create our `panel.Tabs`! The `Tabs` layout is really easy to use. We simply construct it with a list of tuples that pair tab name to tab data. We just do some zipping below to create generic tab names and match them to `panel.Row` layouts, (since we are showing all of the data - current, previous, and delta).

The final key here is specifying the `css_classes` kwarg. Here we are telling `panel` to render the tabs and add the class `panel-hp-bar` to the component. If you inspect the tab component in your browsers element tree you would see that the parent div defining the tabs includes `panel-hp-bar` in its class. The CSS selector that we defined to add the HP Bars is set up to find children of `panel-hp-bar` named `.bk.bk-tab`.

In [5]:
tabs = pn.Tabs(
    *list(
        zip(
            [f'Data {i}' for i in range(len(styled_data))],
            [pn.Row(*td) for td in styled_data]
        )
    ),
    css_classes=['panel-hp-bar']
)
tabs

Tabs(css_classes=['panel-hp-bar'])
    [0] Row
        [0] HTML(Styler)
        [1] HTML(Styler)
        [2] HTML(Styler)
    [1] Row
        [0] HTML(Styler)
        [1] HTML(Styler)
        [2] HTML(Styler)
    [2] Row
        [0] HTML(Styler)
        [1] HTML(Styler)
        [2] HTML(Styler)
    [3] Row
        [0] HTML(Styler)
        [1] HTML(Styler)
        [2] HTML(Styler)
    [4] Row
        [0] HTML(Styler)
        [1] HTML(Styler)
        [2] HTML(Styler)

Here we can see all 5 tabs of data, each with their own colorbar. Since we used uniform distributions, the deltas across each tab are pretty consistent with one another. However, we did modify tabs `Data 3` and `Data 4`, and without even clicking into any tables we can see just by looking at the tab labels that there is something interesting happening in those tables!